In [1]:
### SETTING UP THE DATAFRAME ###

In [395]:
#Importing relevant modules: 
# * dataframe - pandas 
# * graphs - matplotlib 
# * tools - numpy, random, fnmatch, os
# * fasta to dataframe conversion - SeqIO

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import pandas as pd
import random
import collections
import math
from pandas.tools.plotting import table
from itertools import product
from Bio import SeqIO

In [3]:
work_dir = '/home/gamran/dirtreetest/'

In [398]:
#Functions that generate main work directories and subdirectories for:
# * data (all fasta files)
# * blast (all blast output files)
# * analysis (all dataframe summary and graph files)
# When given a top directory path

def create_main_directories(top_dir):
    """Given a top work directory, creates 3 main subdirectories - data, blast and analysis"""
    main_directory_list = ['data/', 'blast/', 'analysis/']
    for new_direct in main_directory_list:
        if not os.path.exists(top_dir + new_direct):
            os.makedirs(top_dir + new_direct)

def create_data_subdirectories(top_dir):
    data_subdirectory_list = ['basecalled/',
                        'rghityes/',
                        'rghitno/',
                        'rghitsamples/']
    data_subsubdirectory_list = ['ncbihityes/',
                                 'ncbihitno/']
    for new_subdirect in data_subdirectory_list:
        if not os.path.exists(top_dir + 'data/' + new_subdirect):
            os.makedirs(top_dir + 'data/' + new_subdirect)
    for new_subsubdirect in data_subsubdirectory_list:
        if not os.path.exists(top_dir + 'data/rghitno/' + new_subsubdirect):
            os.makedirs(top_dir + 'data/rghitno/' + new_subsubdirect)

def create_blast_subdirectories(top_dir):
    blast_subdirectory_list = ['rgbesthit/',
                               'ncbibesthit/']
    for new_subdirect in blast_subdirectory_list:
        if not os.path.exists(top_dir + 'blast/' + new_subdirect):
            os.makedirs(top_dir + 'blast/' + new_subdirect)

def create_analysis_subdirectories(top_dir):
    analysis_subdirectory_list = ['graphs/',
                                  'tables/',
                               'summaries/']
    for new_subdirect in analysis_subdirectory_list:
        if not os.path.exists(top_dir + 'analysis/' + new_subdirect):
            os.makedirs(top_dir + 'analysis/' + new_subdirect)

In [400]:
#Apply previous functions to generate data, blast and analysis directories and subdirectories:
create_main_directories(work_dir)
create_data_subdirectories(work_dir)
create_blast_subdirectories(work_dir)
create_analysis_subdirectories(work_dir)

In [ ]:
#Create a function that moves all basecalled fasta files to the ./basecalled folder, given input

In [89]:
#Generates a list of all the fasta files present in data/basecalled
os.chdir(work_dir + 'data/')
#os.curdir
#os.getcwd()

basecalled_file_names = [x for x in list(os.walk('./basecalled', topdown=False))]

In [88]:
#Reduces basecalled_file_names to a flattened list, extracts all basecalled fasta file names
def flatten(l):
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

basecalled_file_names_flattened = list(flatten(basecalled_file_names))
basecalled_fasta_file_names = [x for x in basecalled_file_names_flattened if '.fa' in x]
basecalled_fasta_file_names.sort()


['1212_1D_barcoding_Wagga_BC01.fa',
 '1212_1D_barcoding_Wagga_BC02.fa',
 '1212_1D_barcoding_Wagga_BC03.fa',
 '1212_1D_barcoding_Wagga_BC04.fa',
 '1212_1D_barcoding_Wagga_BC05.fa',
 '1212_1D_barcoding_Wagga_BC06.fa',
 '1212_1D_barcoding_Wagga_BC07.fa',
 '1212_1D_barcoding_Wagga_BC08.fa',
 '1212_1D_barcoding_Wagga_BC09.fa',
 '1212_1D_barcoding_Wagga_BC10.fa',
 '1212_1D_barcoding_Wagga_BC11.fa',
 '1212_1D_barcoding_Wagga_BC12.fa',
 '1212_1D_barcoding_Wagga_BC13.fa',
 '1212_1D_barcoding_Wagga_BC14.fa',
 '1212_1D_barcoding_Wagga_BC15.fa',
 '1212_1D_barcoding_Wagga_BC16.fa',
 '1212_1D_barcoding_Wagga_BC17.fa',
 '1212_1D_barcoding_Wagga_BC19.fa',
 '1212_1D_barcoding_Wagga_BC20.fa',
 '1212_1D_barcoding_Wagga_BC21.fa',
 '1212_1D_barcoding_Wagga_BC22.fa',
 '1212_1D_barcoding_Wagga_BC23.fa',
 '1212_1D_barcoding_Wagga_BC24.fa',
 '1212_1D_barcoding_Wagga_BC25.fa',
 '1212_1D_barcoding_Wagga_BC26.fa',
 '1212_1D_barcoding_Wagga_BC27.fa',
 '1212_1D_barcoding_Wagga_BC28.fa',
 '1212_1D_barcoding_Wagga_BC

In [92]:
#Make a list of all available barcodes, based on fasta file names
basecalled_fasta_barcodes = []
for bcs in range(0, len(basecalled_fasta_file_names)):
    basecalled_fasta_barcodes.append(basecalled_fasta_file_names[bcs][-7:-3]) #Convert to a search parameter?
#print(basecalled_fasta_file_names[1][-7:-3])  
#print(basecalled_fasta_barcodes)

In [197]:
#Specifies all good barcodes
all_barcodes = basecalled_fasta_barcodes
good_barcodes = ['BC01', 'BC02', 'BC03', 'BC04', 'BC05', 'BC06']
bad_barcodes = sorted(list(set(all_barcodes) - set(good_barcodes)))
#bad_barcodes_BC00 = [x for x in bad_barcodes if 'BC' in x]
#bad_barcodes_NB00 = [x for x in bad_barcodes if 'NB' in x]


In [122]:
#Extracts the ID from the basecalled file names (everything before BC/NB??.fa)
basecalled_fasta_file_id = list(set([x[:-8] for x in basecalled_fasta_file_names]))[0]

In [210]:
#Create text file to summarise read_id, barcode and length for all basecalled reads, save in analysis/summaries/
analysis_summaries_path = work_dir + 'analysis/summaries/'
complete_path_and_name = os.path.join(analysis_summaries_path, basecalled_fasta_file_id + ".lengths.txt")
lengths_txt = open(complete_path_and_name, "w")
print("Read_id,Barcode,Length,Quality", file=lengths_txt)

os.chdir(work_dir + 'data/basecalled/')
for fa_file in basecalled_fasta_file_names: #file_names had the list of all the barcode fasta file names
    for seq in SeqIO.parse(open(fa_file), 'fasta'):
        if fa_file[-7:-3] in good_barcodes:
            print(seq.id + "," + fa_file[-7:-3] + "," + str(len(seq)) + "," + 'Good', file=lengths_txt)
        elif fa_file[-7:-3] in bad_barcodes:
            print(seq.id + "," + 'NB00' + "," + str(len(seq)) + "," + 'Bad', file=lengths_txt)
        else:
            continue
lengths_txt.close()

In [211]:
basecalled_df = pd.read_csv(complete_path_and_name)


In [568]:
#basecalled_df.tail()

In [ ]:
### DATAFRAME ANALYSIS - TABLE OF SUMMARY DATA ###

In [246]:
### Shows number of reads, sum, max, mean, median of read lengths per barcode, all bad barcodes sorted to NB00 ###
basecalled_df_pivot = basecalled_df.pivot_table(values='Length', 
                                                index='Barcode', 
                                                aggfunc=[len, np.sum, np.max, np.mean, np.median],
                                                fill_value=0, 
                                                margins=True)
basecalled_df_pivot.index.name = None

for clmns in ['len', 'amax', 'median']:
    basecalled_df_pivot[clmns] = basecalled_df_pivot[clmns].apply(lambda x: int(x))
basecalled_df_pivot['sum'] = basecalled_df_pivot['sum'].apply(lambda x: "{0:.3f}".format(int(x)/1000000))
basecalled_df_pivot['mean'] = basecalled_df_pivot['mean'].apply(lambda x: "{0:.2f}".format(x))
basecalled_df_pivot_formal = basecalled_df_pivot.copy() #titles have whitespace, make fancier
basecalled_df_pivot_formal.rename(columns={'len': 'Number of Reads', 
                                    'sum': 'Total Length (Mbp)',
                                    'amax': 'Max Length (bp)',
                                    'mean': 'Mean Length (bp)',
                                    'median': 'Median Length (bp)'}, inplace=True)



['Number of Reads',
 'Total Length (Mbp)',
 'Max Length (bp)',
 'Mean Length (bp)',
 'Median Length (bp)']

In [437]:
### Saves pivot table summary in analysis/tables/ folder ###
fig = plt.figure(figsize=(9,2))
ax = fig.add_subplot(1, 1, 1, frame_on=False)
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table(ax, basecalled_df_pivot_formal, loc='center')

plt.savefig(os.path.join(work_dir + 'analysis/tables/', 
                        basecalled_fasta_file_id + "_basecalled_reads_summary.png"))



In [ ]:
### DATAFRAME ANALYSIS - HISTOGRAMS OF SUMMARY DATA ###

In [268]:
#Generates a histogram showing the total read length distribution for all basecalled reads
plt.figure(figsize=(10,5))
basecalled_df.Length.hist(bins=30)

plt.title('Basecalled Reads - Total Read Length Distribution', y=1.03, fontsize='x-large', fontweight='bold')

plt.xlabel('Read Length', fontsize=16)
plt.xticks(np.arange(0, basecalled_df.Length.max() + 1, 2000))
plt.xlim([-1000, basecalled_df.Length.max() + 1000])

plt.ylabel('Read Count', fontsize=16)

for idx, clmn_name in enumerate(list(basecalled_df_pivot_formal.columns)):
    plt.annotate(clmn_name + ' = ' + str(basecalled_df_pivot_formal[clmn_name]['All']), 
                 xy=(1, 1), 
                 xycoords='axes fraction', 
                 fontsize=16, 
                 fontweight='normal',
                 xytext=(-20, -30 - 30*idx), 
                 textcoords='offset points', 
                 ha='right', 
                 va='top')
    
plt.savefig(work_dir + 'analysis/graphs/graph1.png', bbox_inches='tight')

In [359]:
#Generates a histogram showing the total read length distribution for all basecalled reads per barcode

#Assesses number of barcodes, generates necessary number of plots (even) and relevant indexing system
basecalled_barcodes = list(basecalled_df['Barcode'].unique())

if len(list(basecalled_df['Barcode'].unique())) % 2 == 0:
    no_of_subplots = len(basecalled_barcodes)
else:
    no_of_subplots = len(basecalled_barcodes) + 1

#Always 2 columns, bc-count/2 rows 
no_of_subplots_pair = [int(no_of_subplots/2), 2]

#Produce pairs of indices correlating to the coordinates of the subplots
subplot_coordinates = list(product(range(no_of_subplots_pair[0]), range(no_of_subplots_pair[1])))
subplot_coordinates_list = [list(l) for l in subplot_coordinates]
subplot_coordinates_list_rows = [i[0] for i in subplot_coordinates_list]
subplot_coordinates_list_columns = [i[1] for i in subplot_coordinates_list]

fig, ax = plt.subplots(no_of_subplots_pair[0], no_of_subplots_pair[1], figsize=(15,10))

xmax = int(basecalled_df.Length.max())

grouped = basecalled_df.groupby('Barcode')  # CHECK THIS FOR SCALABILITY, NOT ENTIRELY SURE WHAT THIS DOES
max_count_list = []
for key in grouped.groups.keys():
    max_count_list.append(np.histogram(grouped.Length.get_group(key), 50)[0].max())
max_count_list.sort()

def applyGroupHistograms(ax_ind1, ax_ind2, bcs):
    ax[ax_ind1, ax_ind2].hist(basecalled_df.groupby('Barcode')['Length'].get_group(bcs), 
                              bins=range(0, xmax, int(xmax/60)), 
                              color='green', 
                              alpha=0.8)
    ax[ax_ind1, ax_ind2].set_title(bcs)
    ax[ax_ind1, ax_ind2].set_xlabel('Read Length')
    ax[ax_ind1, ax_ind2].set_ylabel('Read Count');
    ax[ax_ind1, ax_ind2].set_xlim([-1000, basecalled_df['Length'].max() + 1000])
    ax[ax_ind1, ax_ind2].set_xticks(np.arange(0, xmax + 1, 2000))
    ax[ax_ind1, ax_ind2].set_ylim(0, max_count_list[-1]*(5/4))
    ax[ax_ind1, ax_ind2].grid(True, which='Major')
    
    for idx, clmn_name in enumerate(['Number of Reads', 'Total Length (Mbp)', 'Median Length (bp)']):
        ax[ax_ind1, ax_ind2].annotate(clmn_name + ' = ' + str(basecalled_df_pivot_formal[clmn_name][bcs]), 
                     xy=(1, 1), 
                     xycoords='axes fraction', 
                     fontsize=16, 
                     fontweight='normal',
                     xytext=(-20, -25 - 30*idx), 
                     textcoords='offset points', 
                     ha='right', 
                     va='top')

for ax_ind1, ax_ind2, bcs, in zip(subplot_coordinates_list_rows, subplot_coordinates_list_columns, bc_list):
    applyGroupHistograms(ax_ind1, ax_ind2, bcs)    

if len(list(basecalled_df['Barcode'].unique())) != 0:
    plt.delaxes(ax[subplot_coordinates_list_rows[-1], subplot_coordinates_list_columns[-1]])

plt.suptitle('Basecalled Reads - Read Length Distribution By Barcode', 
             y=1.03, 
             fontsize='x-large', 
             fontweight='bold')
plt.tight_layout()

plt.savefig(work_dir + 'analysis/graphs/graph2.png', bbox_inches='tight')

In [ ]:
### DATAFRAME ANALYSIS - COLLATION OF RGBESTHIT DATA ###

In [ ]:
#Create a function that moves all blast files to the blast/basecalled folder, given input

In [374]:
os.chdir(work_dir + 'blast/')
rgblast_file_names = [x for x in list(os.walk('./rgbesthit', topdown=False))]
rgblast_file_names_flattened = list(flatten(rgblast_file_names))
rgblast_besthit_file_names = [x for x in rgblast_file_names_flattened if 'blast.besthit' in x]
rgblast_besthit_file_names.sort()

In [377]:
rgblast_besthit_file_names

['1212_1D_barcoding_Wagga_BC00.fa.WW_19121016.blast.besthit',
 '1212_1D_barcoding_Wagga_BC01.fa.WW_19121016.blast.besthit',
 '1212_1D_barcoding_Wagga_BC02.fa.WW_19121016.blast.besthit',
 '1212_1D_barcoding_Wagga_BC03.fa.WW_19121016.blast.besthit',
 '1212_1D_barcoding_Wagga_BC04.fa.WW_19121016.blast.besthit',
 '1212_1D_barcoding_Wagga_BC05.fa.WW_19121016.blast.besthit',
 '1212_1D_barcoding_Wagga_BC06.fa.WW_19121016.blast.besthit',
 '1212_1D_barcoding_Wagga_NB00.fa.WW_19121016.blast.besthit']

In [560]:
os.chdir(work_dir + 'blast/rgbesthit')
rgblast_dfs_dict = {}
for besthit_file in rgblast_besthit_file_names:
    if '.blast.besthit' in besthit_file:
        key = '' #Safety 
        key = besthit_file.split('.')[0][-4:]
        rgblast_dfs_dict[key] = pd.read_table(besthit_file, delim_whitespace=True, 
                                           names=['qseqid', 'sseqid', 'evalue', 'bitscore', 'length', 'pident', 
                                           'nident', 'species', 'barcode'])
for bcs in rgblast_dfs_dict.keys():
    rgblast_dfs_dict[bcs]['barcode'] = bcs

In [561]:
# Must supply two lists of equal length, in the desired order: 
# one corresponding to unique letter combos in the sseqid ids (tags)
# one corresponding to the full names of the reference genomes used
ref_genome_tag_list = ['TGAC', 'Zymo', 'Ptri', 'Pst', 'tig']
ref_genome_species_list = ['Wheat', 
                        'Zymoseptoria tritici', 
                        'Pyrenophore tritici-repentis', 
                        'P. striformis f. sp. tritici WA', 
                        'Parastagonospora nodorum']

def assign_species_from_sseqid(x):
    for rg_tag, rg_full in zip(ref_genome_tag_list, ref_genome_species_list):    
        if rg_tag in str(x):
            return rg_full
    else:
        return 'N/A'

for bcs in rgblast_dfs_dict.keys():
    rgblast_dfs_dict[bcs]['species'] = rgblast_dfs_dict[bcs]['sseqid'].apply(assign_species_from_sseqid)


In [566]:
#Combining the BC00 reads from BLAST
if 'BC00' in rgblast_dfs_dict.keys():
    rgblast_dfs_dict['BC00']['barcode'] = 'NB00'

#Combine all the reference genome hit dataframes across all barcodes
rgblast_df_list = []
for bcs in rgblast_dfs_dict.keys():
    rgblast_df_list.append(rgblast_dfs_dict[bcs])
    
rgblast_df_joint = pd.concat(rgblast_df_list, axis=0, ignore_index=True)
rgblast_df_joint.sort_values('barcode', ascending=True, inplace=True)
rgblast_df_joint.reset_index(drop=True, inplace=True)

In [571]:
#Create a pivot table showing all reference genome hit counts for each barcode
rgblast_df_joint_pivot = rgblast_df_joint.pivot_table(values='nident', index='barcode', columns='species', aggfunc='count', fill_value=0, margins=True)
rgblast_df_joint_pivot = rgblast_df_joint_pivot.applymap(int)
rgblast_df_joint_pivot.index.name = None
rgblast_df_joint_pivot.columns.name = None

In [587]:
#Save pivot table to analysis/tables/
fig = plt.figure(figsize=(11,2))
ax = fig.add_subplot(1, 1, 1, frame_on=False)
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table(ax, rgblast_df_joint_pivot, loc='center')

plt.savefig(os.path.join(work_dir + 'analysis/tables/', 
                        basecalled_fasta_file_id + "_rgblasthit_reads_summary.png"))


In [573]:
#Removes the 'all' column (for graphing purposes)
rgblast_df_joint_pivot_noall = rgblast_df_joint_pivot.drop('All', axis=1)

In [803]:
# Construct a grouped bar chart showing the reference genome hit counts per barcode (log y axis)
def generateGBGforRGHC(dropmaxspecies=False):
    n_groups = len(rgblast_df_joint_pivot_noall.index.values)

    max_genome = str(rgblast_df_joint_pivot_noall.ix['All'].idxmax(axis=1))

    spc_list = list(rgblast_df_joint_pivot_noall.columns)
    
    if dropmaxspecies == True:
        spc_list.remove(max_genome)

    n_species_dict = {}
    for spc in spc_list:
        n_species_dict[spc] = tuple(rgblast_df_joint_pivot_noall[spc])

    n_nohit = tuple(basecalled_df_pivot['len'].values - rgblast_df_joint_pivot['All'].values)    
    
    fig, ax = plt.subplots(figsize=(7,3))

    index = np.arange(n_groups)
    bar_width = 0.13
    opacity = 0.4

    cycol = 'rgbcym'
    if dropmaxspecies == False:
        plt.bar(index - bar_width*2, 
            n_nohit, 
            bar_width,
            alpha = opacity,
            log = True,
            color = 'k',
            label = 'No Hit')

        for idx, spc in enumerate(spc_list):
            plt.bar(index - bar_width + bar_width*idx, 
                n_species_dict[spc], 
                bar_width,
                alpha = opacity,
                log = True,
                color = random.choice(cycol),
                label = spc)
    else:
        for idx, spc in enumerate(spc_list):
            plt.bar(index - bar_width + bar_width*idx, 
                n_species_dict[spc], 
                bar_width,
                alpha = opacity,
                color = random.choice(cycol),
                label = spc)
            
    def generate_upper_ylim(graph_max):
        """Takes a value, counts the digits and outputs a value corresponding to 10*digits"""
        digits = len(str(graph_max))
        round_up = 10**(digits)
        return round_up

    plt.xlabel('Barcode')
    plt.xticks(index + bar_width*1.2, tuple(rgblast_df_joint_pivot_noall.index.values))

    plt.ylabel('Number of Reads')
    if dropmaxspecies == False:
        plt.ylim(0, generate_upper_ylim(rgblast_df_joint_pivot['All'].values.max()))
    else:
        rgblast_df_joint_pivot_nomaxspecies = rgblast_df_joint_pivot_noall.copy()
        rgblast_df_joint_pivot_nomaxspecies.drop('Wheat', axis=1, inplace=True)
        rgblast_df_joint_pivot_nomaxspecies['All'] = rgblast_df_joint_pivot_nomaxspecies.sum(axis=1)
        plt.ylim(0, (rgblast_df_joint_pivot_nomaxspecies['All'].values.max()+1)/2)

    plt.title('Number of Reads Hitting Specific Reference Genomes', y=1.08)
    plt.legend(loc='center left', bbox_to_anchor=(1.01, 0.5), fontsize=10)
    plt.grid(b=None, which='major', axis='y', linestyle='dashed', linewidth=0.2)
    plt.margins(x=0.1)
    plt.tick_params(top="off", right="off", bottom='off')
    plt.tight_layout()
    if dropmaxspecies == False:
        plt.savefig(work_dir + 'analysis/graphs/graph3.png', bbox_inches='tight')
    else:
        plt.savefig(work_dir + 'analysis/graphs/graph4.png', bbox_inches='tight')

In [804]:
generateGBGforRGHC(dropmaxspecies=False)

In [805]:
generateGBGforRGHC(dropmaxspecies=True)

In [ ]:
#graphs to generate: The same without the max species (in this case wheat): -(dropmaxgenome=True)

#Make same graphs, but with percentages

0         TGACv1_scaffold_327975_4BS
1         TGACv1_scaffold_030242_1BL
2         TGACv1_scaffold_606415_7DL
3         TGACv1_scaffold_031052_1BL
4         TGACv1_scaffold_602887_7DL
5          TGACv1_scaffold_234012_3B
6         TGACv1_scaffold_499976_6BL
7          TGACv1_scaffold_227006_3B
8         TGACv1_scaffold_527989_6DL
9         TGACv1_scaffold_609188_7DL
10        TGACv1_scaffold_556005_7AL
11        TGACv1_scaffold_320259_4BL
12                    000006F_Pst_WA
13        TGACv1_scaffold_069514_1DL
14        TGACv1_scaffold_006707_1AL
15        TGACv1_scaffold_177600_2DS
16        TGACv1_scaffold_499980_6BL
17        TGACv1_scaffold_405113_5BL
18        TGACv1_scaffold_569322_7AS
19        TGACv1_scaffold_375334_5AL
20        TGACv1_scaffold_177638_2DS
21        TGACv1_scaffold_081237_1DS
22        TGACv1_scaffold_569353_7AS
23        TGACv1_scaffold_294933_4AL
24        TGACv1_scaffold_543862_6DS
25        TGACv1_scaffold_592143_7BS
26        TGACv1_scaffold_273638_3DS
2